## Лаба 6. Подсчёт статистики по фильмам

### Дедлайн⏰ 

Четверг, 23 мая 2019 года, 23:59.

### Задача

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

#### Обработка данных на вход

Имеются следующие входные данные:

* Таблица users * films с рейтингами. Архив с датасетом нужно скачать с сайта GroupLens: http://files.grouplens.org/datasets/movielens/ml-100k.zip. Также, он загружен на HDFS в /labs/lab06data/ml-100k. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.
* id фильма для расчета индивидуальных характеристик — в вашем Личном кабинете на странице Лабы 6.

#### Обработка данных на выход

Для выданного id фильма:
1. Построить распределения оценок. Получится таблица, где в первом столбце стоят значения оценки 1,2,3,4,5, а во втором — количество человек, поставивших соответствующую оценку. Поле должно называться “hist_film”.
2. Чтобы понять какие оценки предпочитают ставить пользователи MovieLens, постройте аналогичное предыдущему пункту распределение оценок, но только уже для всего датасета. Поле должно называться “hist_all”.

Файл с решением должен содержать только значения распределения оценок 1,2,3,4,5 из вашей полученной таблицы.

В поле “hist_film” нужно указать в следующем порядке для заданного id фильма количество поставленных оценок: 1,2,3,4,5.

В поле “hist_all” нужно указать в следующем порядке для всех фильмов общее количество поставленных оценок: 1,2,3,4,5.

Выходной формат — json. Пример решения:

`{
"hist_film": [134,123,782,356,148],
"hist_all": [134,123,782,356,148]
}`

#### Проверка

Файл необходимо положить в свою домашнюю директорию под названием: lab06.json.

Проверка осуществляется автоматическим скриптом из Личного кабинета.

## Решение

### Load dataset

In [1]:
import pandas as pd
import json

In [10]:
# Load the main table with users, films, ratings
df = pd.read_csv('ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

In [11]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [12]:
df.user_id.nunique()

943

In [13]:
df.item_id.nunique()

1682

In [14]:
df.rating.unique()

array([3, 1, 2, 4, 5])

In [117]:
# Load films description
df2 = pd.read_csv('ml-100k/u.item', sep='\|', header=None, names=['movie_id', 'movie_title', 'release date', 
                                                                  'video release date', 'IMDb URL', 'unknown', 
                                                                  'Action', 'Adventure', 'Animation', 'Children', 
                                                                  'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 
                                                                  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 
                                                                  'Sci-Fi', 'Thriller', 'War', 'Western'], 
                  index_col=0, usecols=[0,1], engine='python')

df2.head()    

,movie_title
movie_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [120]:
df2.shape

(1682, 1)

### Calculate distribution of ratings
#### film id for calculations = 257

In [17]:
# распределение оценок для выбранного фильма
hist_film = df.loc[df['item_id']==257, ['user_id', 'rating']].groupby('rating').count()
hist_film

,user_id
rating,
1,2
2,28
3,81
4,126
5,66


In [18]:
# распределение оценок для всего датасета
hist_all = df.loc[:, ['user_id', 'rating']].groupby('rating').count()
hist_all

,user_id
rating,
1,6110
2,11370
3,27145
4,34174
5,21201


### Save the distributions to json file

In [131]:
# Make a dictionary to present the distributions in the required format
d = {
'hist_film': [item for item in hist_film.user_id],
'hist_all': [item for item in hist_all.user_id]
}
d

{'hist_film': [2, 28, 81, 126, 66],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [20]:
# Preview json string
json.dumps(d)

'{"hist_film": [2, 28, 81, 126, 66], "hist_all": [6110, 11370, 27145, 34174, 21201]}'

In [8]:
# Save the dictionary to a json file
with open('lab06.json', 'w') as f:
    json.dump(d, f)

## ✨ Лаба 6. Суперачивка. Построение неперсонализированной рекомендательной системы для фильмов

### Дедлайн

⏰ Четверг, 23 мая 2018 года, 23:59.

### Задача

По данным о рейтингах фильмов из MovieLens рекомендовать топ-10 фильмов по разным критериям.

#### Обработка данных на вход

Имеются следующие входные данные:
* Та же таблица `users * films` с рейтингами, что и в [Лабе 6](lab06.md).
* Параметр `k` = 50 для расчета поправленного среднего рейтинга. Параметр берётся из [Личного кабинета](http://lk.newprolab.com/lab/laba06s).
* Процент доверия для расчёта границ доверительного интервала [1.645, u'90.00%']. Параметр берётся из [Личного кабинета](http://lk.newprolab.com/lab/laba06s).

#### Обработка данных на выход

Для каждого фильма необходимо посчитать:
1. Количество человек `n`, поставивших рейтинг фильму .
2. Средний рейтинг фильма (`сумма рейтингов фильма / количество человек, оценивших фильм`) 

![tex1](images/lab06s_eq1.svg)

3. Количество человек `m`, оценивших фильм положительно. Оценки 4 и выше  считаются положительными. 
4. Доля людей, оценивших фильм положительно (`пункт 3 / пункт 1` или `m / n`).
5. Глобальное среднее по всему датасету. `Сумма всех оценок по всем фильмам /Количество всех оценок по всем фильмам`.

![tex2](images/lab06s_eq2.svg)

6. Оценку, поправленную на нехватку данных:

![tex3](images/lab06s_eq3.svg)

Мы искусственно добавляем `k` глобальных средних (\mu из пункта 5) каждому фильму.

7. Нижнюю и верхнюю границы доверительного интервала оценки (Wilson score interval) из лекции с заданным уровнем доверия. 

![tex4](images/lab06s_eq4.svg)

где  `n` —  количество рейтингов (пункт 1), `p` - доля людей, оценивших фильм положительно (пункт 4).
   
| **уровень доверия, %** | 99.9  | 99    | 95    | 90    |
| ---------------------- | ----- | ----- | ----- | ----- |
| **z**                  | 3.291 | 2.576 | 1.960 | 1.645 |

Рекомендовать топ-10 фильмов (**если рейтинги совпадают, то сортировать по алфавиту названий фильмов от A до Z**):

1. По откликам (пункт 1) — поле `“top10_rates”`.
2. По среднему рейтингу (пункт 2) — поле `“top10_average”`.
3. По среднему рейтингу с регуляризацией `k` (пункт 6) — поле `“top10_rating”`.
4. По нижней границе доверительного интервала Wilson (пункт 7) — поле `“top10_lower”`.


В поле `“top10_rates”` нужно указать `id` 10 фильмов, упорядоченных по убыванию числа количества человек, посмотревших фильм.

В поле `“top10_average”` нужно указать `id` 10 фильмов, упорядоченных по среднему рейтингу.

В поле `“top10_rating”` нужно указать `id` 10 фильмов, упорядоченных по среднему рейтингу с регуляризацией.

В поле `“top10_lower”` нужно указать id `10` фильмов, упорядоченных по нижней границе доверительного интервала.

Выходной формат — json. Пример решения:

```
{  
   "top10_rates": [13456, 12378, 78213, ...],
   "top10_average": [13456, 12378, 78213, ...],
   "top10_rating": [13456, 12378, 78213, ...],
   "top10_lower": [13456, 12378, 78213, ...]
}
```

### Проверка

Файл необходимо положить в свою домашнюю директорию под названием: `lab06s.json`.

Проверка осуществляется [автоматическим скриптом](http://lk.newprolab.com/lab/laba06s) из Личного кабинета.

## Решение


In [24]:
# Let's make a matrix user_id x item_id for further caclulations.
matrix = df.pivot(index='user_id', columns='item_id', values='rating')
matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1. Количество человек n, поставивших рейтинг фильму

In [65]:
n = matrix.count(axis=0)
n.head()

item_id
1    452
2    131
3     90
4    209
5     86
dtype: int64

#### 2. Средний рейтинг фильма (сумма рейтингов фильма / количество человек, оценивших фильм)

In [66]:
(matrix.sum()/matrix.count()).head()

item_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
dtype: float64

#### 3. Количество человек m, оценивших фильм положительно. Оценки 4 и выше считаются положительными.

In [67]:
m = matrix[matrix > 3].count()
m.head()

item_id
1    321
2     51
3     34
4    122
5     39
dtype: int64

#### 4. Доля людей, оценивших фильм положительно (пункт 3 / пункт 1 или m / n).

In [68]:
p = m/n
p.head()

item_id
1    0.710177
2    0.389313
3    0.377778
4    0.583732
5    0.453488
dtype: float64

#### 5. Глобальное среднее по всему датасету. Сумма всех оценок по всем фильмам / Количество всех оценок по всем фильмам.

In [57]:
mu = matrix.sum().sum() / n.sum()
mu

3.52986

#### 6. Оценку, поправленную на нехватку данных: Мы искусственно добавляем k глобальных средних ( \mu из пункта 5) каждому фильму.

In [69]:
k = 50
((matrix.sum() + k*mu) / (matrix.count() + k)).head()

item_id
1    3.843612
2    3.295541
3    3.210664
4    3.546305
5    3.385978
dtype: float64

#### 7. Нижнюю и верхнюю границы доверительного интервала оценки (Wilson score interval) из лекции с заданным уровнем доверия:
![tex4](images/lab06s_eq4.svg)
#### где  n — количество рейтингов (пункт 1), p - доля людей, оценивших фильм положительно (пункт 4).

In [87]:
z = 1.645
upper = (p + z**2/(2*n) + z*(p*(1-p)/n + z**2/(4*n**2))**0.5)/(1+z**2/n)
lower = (p + z**2/(2*n) - z*(p*(1-p)/n + z**2/(4*n**2))**0.5)/(1+z**2/n)
lower[:5], upper[:5]

(item_id
 1    0.673905
 2    0.322151
 3    0.298436
 4    0.526921
 5    0.367945
 dtype: float64, item_id
 1    0.743947
 2    0.460956
 3    0.464255
 4    0.638403
 5    0.541869
 dtype: float64)

### Рекомендовать топ-10 фильмов (если рейтинги совпадают, то сортировать по алфавиту названий фильмов от A до Z):

#### 1. По откликам (пункт 1) — поле “top10_rates”. 

In [78]:
top10_rates = n.nlargest(10)
top10_rates

item_id
50     583
258    509
100    508
181    507
294    485
286    481
288    478
1      452
300    431
121    429
dtype: int64

In [83]:
# В поле “top10_rates” нужно указать id 10 фильмов, упорядоченных по убыванию числа количества человек, посмотревших фильм.
top10_rates.index

Int64Index([50, 258, 100, 181, 294, 286, 288, 1, 300, 121], dtype='int64', name='item_id')

#### 2. По среднему рейтингу (пункт 2) — поле “top10_average”.

In [129]:
top10_average = (matrix.sum()/matrix.count()).nlargest(10)   #sort_values(ascending=False)[:11]
top10_average = pd.DataFrame(top10_average, columns=['average']) 
top10_average = top10_average.join(df2).sort_values('movie_title')
top10_average

,average,movie_title
item_id,,
1536,5.0,Aiqing wansui (1994)
1653,5.0,Entertaining Angels: The Dorothy Day Story (1996)
814,5.0,"Great Day in Harlem, A (1994)"
1201,5.0,Marlene Dietrich: Shadow and Light (1996)
1189,5.0,Prefontaine (1997)
1467,5.0,"Saint of Fort Washington, The (1993)"
1500,5.0,Santa with Muscles (1996)
1599,5.0,Someone Else's America (1995)
1293,5.0,Star Kid (1997)


In [130]:
# В поле “top10_average” нужно указать id 10 фильмов, упорядоченных по среднему рейтингу.
top10_average.index

Int64Index([1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122], dtype='int64', name='item_id')

#### 3. По среднему рейтингу с регуляризацией k (пункт 6) — поле “top10_rating”

In [92]:
top10_rating = ((matrix.sum() + k*mu) / (matrix.count() + k)).nlargest(10)
top10_rating

item_id
318    4.331876
64     4.307787
483    4.298611
50     4.293038
12     4.250767
603    4.221981
98     4.203393
127    4.201929
408    4.194401
169    4.187458
dtype: float64

In [86]:
# В поле “top10_rating” нужно указать id 10 фильмов, упорядоченных по среднему рейтингу с регуляризацией.
top10_rating.index

Int64Index([318, 64, 483, 50, 12, 603, 98, 127, 408, 169], dtype='int64', name='item_id')

#### 4. По нижней границе доверительного интервала Wilson (пункт 7) — поле “top10_lower”.

In [90]:
top10_lower = lower.nlargest(10)
top10_lower

item_id
64     0.867956
479    0.862714
318    0.855785
98     0.852515
483    0.851348
603    0.843865
427    0.840588
114    0.836042
408    0.835188
50     0.833999
dtype: float64

In [91]:
# В поле “top10_lower” нужно указать id 10 фильмов, упорядоченных по нижней границе доверительного интервала.
top10_lower.index

Int64Index([64, 479, 318, 98, 483, 603, 427, 114, 408, 50], dtype='int64', name='item_id')

### Save results to a json file

In [134]:
# Make a dictionary for results in the required format
ds = {
"top10_rates": [item for item in top10_rates.index],
"top10_average": [item for item in top10_average.index],
"top10_rating": [item for item in top10_rating.index],
"top10_lower": [item for item in top10_lower.index]
}
ds

{'top10_rates': [50, 258, 100, 181, 294, 286, 288, 1, 300, 121],
 'top10_average': [1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122],
 'top10_rating': [318, 64, 483, 50, 12, 603, 98, 127, 408, 169],
 'top10_lower': [64, 479, 318, 98, 483, 603, 427, 114, 408, 50]}

In [135]:
# Preview json string
json.dumps(ds)

'{"top10_rates": [50, 258, 100, 181, 294, 286, 288, 1, 300, 121], "top10_average": [1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122], "top10_rating": [318, 64, 483, 50, 12, 603, 98, 127, 408, 169], "top10_lower": [64, 479, 318, 98, 483, 603, 427, 114, 408, 50]}'

In [136]:
# Save the dictionary to a json file
with open('lab06s.json', 'w') as f:
    json.dump(ds, f)